## Multiscale analysis of spatial correlation between 2 signals

In [ ]:
from tapenade.analysis.spatial_correlation import (
    SpatialCorrelationPlotter
)
from pathlib import Path
import tifffile
from tapenade.preprocessing._preprocessing import change_array_pixelsize,normalize_intensity

### 1. Data loading

Load your the two signals that you wish to study the correlation between. If raw image data is loaded, the correlation will be studied at the smallest scale (voxel). To study the correlation at larger scales, we recommend applying our masked gaussian filter method, as presented in the `masked_gaussian_smoothing` notebook.

In [ ]:
folder = Path(globals()['_dh'][0]).parents[1] / 'data'
path_to_signal = Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/data/1.tif'
path_to_mask =  Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/masks/1_mask.tif'
path_to_labels = Path(folder) / '2k_Hoechst_FoxA2_Oct4_Bra_78h/big/segmentation/1_seg.tif'
mask = tifffile.imread(path_to_mask)
labels = tifffile.imread(path_to_labels)
signal = tifffile.imread(path_to_signal)
hoechst = signal[:,0,:,:]
foxa2 = signal[:,1,:,:]
oct4 = signal[:,2,:,:]
bra = signal[:,3,:,:]

In [ ]:
hoechst = change_array_pixelsize(hoechst, input_pixelsize=(1,0.6,0.6))
bra = change_array_pixelsize(bra, input_pixelsize=(1,0.6,0.6))
oct4 = change_array_pixelsize(oct4, input_pixelsize=(1,0.6,0.6))
foxa2 = change_array_pixelsize(foxa2, input_pixelsize=(1,0.6,0.6))
mask = (change_array_pixelsize(mask, input_pixelsize=(1,0.6,0.6),order=0)).astype(int)
labels = (change_array_pixelsize(labels, input_pixelsize=(1,0.6,0.6),order=0)).astype(int)

hoechst_norm = normalize_intensity(hoechst,ref_image=hoechst,image_wavelength=405,sigma=11)
foxa2_norm = normalize_intensity(foxa2,ref_image=hoechst,image_wavelength=488,sigma=11)
oct4_norm = normalize_intensity(oct4,ref_image=hoechst,image_wavelength=555,sigma=11)
bra_norm = normalize_intensity(bra,ref_image=hoechst,image_wavelength=647,sigma=11)

In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_image(hoechst_norm*mask,name='hoechst',colormap='gray_r')
viewer.add_image(foxa2_norm*mask,name='foxa2',colormap='gray_r')
viewer.add_image(oct4_norm*mask,name='oct4',colormap='gray_r')
viewer.add_image(bra_norm*mask,name='bra',colormap='gray_r')
viewer.add_labels(labels,name='labels')
viewer.grid.enabled = True
napari.run()

### 2. Initializing the plotter

Initializing the plotter once allows you to test several parameters of the visualization without reprocessing the data.

In [ ]:
spatial_correlation_plotter_foxa2_oct4 = SpatialCorrelationPlotter(
    quantity_X=foxa2_norm,
    quantity_Y=oct4_norm,
    mask=mask,
    labels=labels,

)   
spatial_correlation_plotter_bra_oct4 = SpatialCorrelationPlotter(
    quantity_X=bra_norm,
    quantity_Y=oct4_norm,
    mask=mask,
    labels=labels,

)   
spatial_correlation_plotter_bra_foxa2 = SpatialCorrelationPlotter(
    quantity_X=bra_norm,
    quantity_Y=foxa2_norm,
    mask=mask,
    labels=labels,
)

Plot the correlation:

In [ ]:
lim_bra=1500
lim_foxa2=300
lim_oct4=40
bins=50
fig, ax = spatial_correlation_plotter_foxa2_oct4.get_heatmap_figure(
    show_linear_fit=False,
    show_individual_cells=True,
    display_quadrants=True,
    bins=[bins,bins],
    extent_X = (0,lim_foxa2),
    extent_Y = (0,lim_oct4),
    label_X='FoxA2',
    label_Y='Oct4',
)
fig, ax = spatial_correlation_plotter_bra_oct4.get_heatmap_figure(
    show_linear_fit=False,
    show_individual_cells=True,
    display_quadrants=True,
    bins=[bins,bins],
    extent_X = (0,lim_bra),
    extent_Y = (0,lim_oct4),
    label_X='T-Bra',
    label_Y='Oct4',
)
fig, ax = spatial_correlation_plotter_bra_foxa2.get_heatmap_figure(
    show_linear_fit=False,
    show_individual_cells=True,
    display_quadrants=True,
    bins=[bins,bins],
    extent_X = (0,lim_bra),
    extent_Y = (0,lim_foxa2),
    label_X='T-Bra',
    label_Y='FoxA2',
)
fig.show()
